## Description
Running this notebook will create the file `locast.db` in the notebooks folder (if not yet present), containing a price candle cluster for ETH-USD.
Note: All examples share the same `locast.db`. So if one creates it, the other one will use it automatically. 
Once it exists, re-running this notebook will attempt to update that cluster. Depending on how much time has passed, this might not be necessary yet.



In [ ]:
from datetime import timedelta
import time
from dydx3 import Client  # type: ignore
from dydx3.constants import API_HOST_MAINNET  # type: ignore
from locast.candle_fetcher.dydx.api_fetcher.dydx_v3_fetcher import DydxV3Fetcher
from sqlalchemy import create_engine  # type: ignore


from locast.candle_fetcher.dydx.candle_fetcher.dydx_candle_fetcher import (
    DydxCandleFetcher,
)
from locast.store_manager.store_manager import StoreManager

from locast.candle.candle_utility import CandleUtility as cu
from locast.candle.dydx.dydx_resolution import DydxResolution
from locast.candle.exchange import Exchange
from locast.candle_storage.sql.sqlite_candle_storage import SqliteCandleStorage

DELETE_CLUSTER = False


async def dydx_v3_example() -> None:
    # Create store manager.
    engine = create_engine("sqlite:///locast.db")
    candle_storage = SqliteCandleStorage(engine, log_progress=True)
    dydx_v3_fetcher = DydxV3Fetcher(Client(host=API_HOST_MAINNET))
    candle_fetcher = DydxCandleFetcher(dydx_v3_fetcher, log_progress=True)

    manager = StoreManager(candle_fetcher=candle_fetcher, candle_storage=candle_storage)

    # Define desired candle cluster parameters.
    exchange = Exchange.DYDX
    resolution = DydxResolution.ONE_MINUTE
    market = "ETH-USD"

    # Define start date for candle cluster.
    now = cu.normalized_now(resolution)  # Now, rounded down to nearest resolution tick
    start_date = now - timedelta(seconds=resolution.seconds * 10000)  # 20K candles back

    # Check for existing cluster.
    eth_info = await manager.get_cluster_info(exchange, market, resolution)

    # If cluster head not present (meaning: There is no candle cluster in the database for this market),
    # then create cluster (download from exchange & store in database).
    newly_created = False
    if not eth_info.head:
        print("🛠️ Creating cluster...")
        start = time.time()
        await manager.create_cluster(market, resolution, start_date)
        print(f"✅ Cluster created in {round(time.time() - start, 2)} seconds.")
        newly_created = True

    # Check if cluster is up to date (for example if this notebook is run again later).
    # If not, update cluster.
    eth_info = await manager.get_cluster_info(exchange, market, resolution)
    if not eth_info.is_uptodate:
        print("\n🔄 Updating cluster...")
        start = time.time()
        await manager.update_cluster(exchange, market, resolution)
        print(f"✅ Cluster updated in {round(time.time() - start, 2)} seconds.")
    elif not newly_created:
        await manager.update_cluster(exchange, market, resolution)

    # Check how many candles there are in this cluster
    eth_info = await manager.get_cluster_info(exchange, market, resolution)
    print(f"🕯️{market}-Candles in cluster: {eth_info.amount}.")

    # Set DELETE flag at the top to True, in order to try out deleting the eth candle cluster
    if DELETE_CLUSTER:
        print("\n🗑️ Deleting cluster...")
        start = time.time()
        await manager.delete_cluster(exchange, market, resolution)
        print(f"✅ Cluster deleted in {round(time.time() - start, 2)} seconds.")
        if cluster_info := await manager.get_cluster_info(exchange, market, resolution):
            print(f"🕯️{market}-Candles in cluster: {cluster_info.amount}.")


await dydx_v3_example()